This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/trajectories.html).  I recommend having both windows open, side-by-side!

In [1]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (AddMultibodyPlantSceneGraph, Box, Cylinder,
                         DiagramBuilder, InverseKinematics, MeshcatVisualizer,
                         MeshcatVisualizerParams, RigidTransform, RollPitchYaw,
                         RotationMatrix, Solve, StartMeshcat,
                         Sphere, Rgba, JointSliders)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders
from manipulation.scenarios import AddIiwa, AddShape, AddWsg


In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Interactive inverse kinematics

We use this to get the joint angles for the target positions

In [3]:
def get_joints_inverse_kinematics(X_WTarget):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddIiwa(plant, "with_box_collision")
    wsg = AddWsg(plant, iiwa, welded=True)
    plant.Finalize()

    meshcat.DeleteAddedControls()
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, 
        scene_graph, 
        meshcat,
        MeshcatVisualizerParams(prefix="Spatial"))
    joint_sliders = builder.AddSystem(JointSliders(meshcat, plant))

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)

    def my_callback(context, pose):
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
            gripper_frame, [0, 0.07, 0], plant.world_frame(),
            pose.translation(), pose.translation())
        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(), plant.world_frame(),
            pose.rotation(), 0.0)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        prog = ik.get_mutable_prog()
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
        prog.SetInitialGuess(q, q0)
        result = Solve(ik.prog())
        clear_output(wait=True)
        if result.is_success():
            print("IK success")
            joint_sliders.SetPositions(result.GetSolution())
            print(f"Joint positions: {result.GetSolution()}")
        else:
            print("IK failure")

    sliders = MeshcatPoseSliders(meshcat)
    sliders.SetPose(plant.EvalBodyPoseInWorld(
        plant_context, plant.GetBodyByName("body", wsg)))
    # sliders.SetXyz(X_WTarget.translation())
    sliders.SetPose(X_WTarget)
    sliders.Run(visualizer, context, my_callback)

X_WStart = RigidTransform([0.5, 0, 0.15])
meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(.9, .1, .1, 1))
meshcat.SetTransform("start", X_WStart)
X_WGoal = RigidTransform(RollPitchYaw([-0.16, 3.14, -1.92]),[0, -0.6, 0.15])
meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(.1, .9, .1, 1))
meshcat.SetTransform("goal", X_WGoal)

get_joints_inverse_kinematics(X_WGoal)
# get_joints_inverse_kinematics(X_WStart)


IK success
Joint positions: [-2.00188489  1.23067234  0.03573024 -1.42203902  1.57121904  1.71564864
 -2.64577454]


KeyboardInterrupt: 

This one has the hand tracking a cylinder, but is allowed to touch anywhere along the cylinder.  The sliders are controlling the pose of the cylinder. Or you can set `grasp_cylinder` to `False` and just chase the robot around with a stick.

In [4]:
meshcat.Delete()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ec511a9b-67ea-463d-b7e3-c38e1afc222b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>